In [1]:
import numpy as np

from bwb.distributions import DistributionDrawDataLoader
from bwb.distributions import ExplicitPosteriorPiN

expl_posterior = ExplicitPosteriorPiN()
expl_posterior

In [2]:
models_array = np.load(r"..\data\face.npy")
models_array = models_array / np.sum(models_array, 1).reshape(-1, 1)
models_array.sum(1)

array([1., 1., 1., ..., 1., 1., 1.])

In [3]:
%%time

models_array = np.load(r"..\data\face.npy")
n_faces, _ = models_array.shape
print(f"{n_faces = }")

faces = DistributionDrawDataLoader(models_array, (28, 28))#.create_from_array(models_array[:10_000])

n_faces = 161666
CPU times: total: 7.73 s
Wall time: 2.27 s


In [4]:
faces[0]

In [5]:
x = faces[0].sample((10,))
x

tensor([124, 479, 288, 534, 473, 159, 423, 356, 501, 690])

In [6]:
%%time
expl_posterior.fit(x, faces)

CPU times: total: 188 ms
Wall time: 51 ms


In [7]:
expl_posterior.rvs(size=10_000, seed=42)

In [8]:
expl_posterior.total_time

0.37649370000872295

In [9]:
expl_posterior2 = ExplicitPosteriorPiN()
expl_posterior2.total_time, expl_posterior.total_time

(0.0, 0.37649370000872295)

In [10]:
try:
    expl_posterior2.draw()
except Exception as e:
    print(e)

This ExplicitPosteriorPiN instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.


In [11]:
len(expl_posterior.samples_counter)

674

In [12]:
print(expl_posterior.samples_counter.total())
expl_posterior.draw()
print(expl_posterior.samples_counter.total())

10000
10001
